In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import confusion_matrix

from src.utils import calculate_profit, train_val_test_split

## Construct the `profit` and `profitable` variables

In [2]:
funnel = pd.read_csv('train_data/funnel.csv')
funnel['profit'] = calculate_profit(funnel)
funnel['profitable'] = (funnel['profit'] > 0).astype(int)

train_funnel, val_funnel, test_funnel = train_val_test_split(funnel, (0.8, 0.1, 0.1))

## Construct balances

I'll try to get last balance, and then difference from last balance for last few months

In [3]:
balances = pd.read_csv('train_data/balance.csv')

In [4]:
# Sum up across all accounts
account_sums = balances.groupby(['client_id', 'month_end_dt'])[['avg_bal_sum_rur', 'max_bal_sum_rur', 'min_bal_sum_rur']].sum()

# Get average range (max - min) for all clients
account_sums['range'] = account_sums['max_bal_sum_rur'] - account_sums['min_bal_sum_rur']
avg_range = account_sums['range'].mean(level='client_id', skipna=True)

# Get the average amount in the last month
last_month_avg = account_sums.iloc[account_sums.index.get_level_values('month_end_dt') == '2019-08-31']
last_month_avg = last_month_avg['avg_bal_sum_rur']
last_month_avg.index = last_month_avg.index.droplevel('month_end_dt')

# Get differences from the average in the last month
account_sums['diff_last_month'] = account_sums['avg_bal_sum_rur'] - last_month_avg
diffs = account_sums['diff_last_month'].reset_index().query('month_end_dt != "2019-08-31"')
diffs = diffs.pivot(index='client_id', columns='month_end_dt')

# Put together all balance features
diffs.columns = [f'balance_diff_{c[1]}' for c in diffs.columns]
balance_ft = diffs
balance_ft['avg_range'] = avg_range
balance_ft['last_month_avg'] = last_month_avg

In [5]:
balance_ft.describe()

,balance_diff_2018-09-30,balance_diff_2018-10-31,balance_diff_2018-11-30,balance_diff_2018-12-31,balance_diff_2019-01-31,balance_diff_2019-02-28,balance_diff_2019-03-31,balance_diff_2019-04-30,balance_diff_2019-05-31,balance_diff_2019-06-30,balance_diff_2019-07-31,avg_range,last_month_avg
count,16455.000000,16710.000000,17056.000000,17412.000000,17633.000000,17942.000000,18271.000000,18423.000000,18582.000000,18605.000000,18618.000000,18619.000000,18619.000000
mean,87.988210,51.673190,46.710483,33.441822,17.955481,10.769981,4.449948,8.166042,5.379130,6.465090,4.418520,52.000720,-90.814759
std,1884.751453,1181.022633,1157.721772,950.131726,553.243603,547.045252,498.944701,476.696726,352.540639,324.753785,262.595122,172.862765,934.295467
min,-38052.000000,-25178.000000,-24428.000000,-23909.000000,-23319.000000,-22554.000000,-21500.000000,-21311.000000,-21300.000000,-21454.000000,-20211.000000,0.000000,-31173.000000
25%,-5.000000,-7.000000,-8.000000,-8.000000,-10.000000,-11.000000,-13.000000,-10.000000,-8.000000,-5.000000,-2.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.833333,0.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.833333,7.000000
max,108444.000000,77117.000000,77565.000000,76899.000000,11144.000000,13059.000000,12013.000000,22115.000000,11144.000000,11144.000000,5795.000000,7792.083333,38052.000000


## Construct client dataset

In [6]:
client = pd.read_csv('train_data/client.csv')

In [7]:
# Take out citizenship and job_type, they are useless
client_ft = client.set_index('client_id')[['gender', 'age', 'region', 'city', 'education']]

# Filter out cities and region to only those above 200, make them categorical (not numerical)
region_counts = client_ft['region'].value_counts(dropna=False)
top_regions = region_counts[region_counts > 200].index

city_counts = client_ft['city'].value_counts(dropna=False)
top_cities = city_counts[city_counts > 200].index

client_ft.loc[~ client_ft['city'].isin(top_cities), 'city'] = None
client_ft.loc[~ client_ft['region'].isin(top_regions), 'region'] = None

client_ft['city'] = client_ft['city'].astype('str')
client_ft['region'] = client_ft['region'].astype('str')

client_ft[['gender', 'education']] = client_ft[['gender', 'education']].fillna('nan')

In [8]:
client_ft.describe(include='all')

,gender,age,region,city,education
count,21498,21495.000000,21498,21498,21498
unique,3,NaN,29,17,8
top,F,NaN,nan,nan,nan
freq,11116,NaN,4461,13862,12218
mean,NaN,43.097139,NaN,NaN,NaN
std,NaN,10.798968,NaN,NaN,NaN
min,NaN,21.000000,NaN,NaN,NaN
25%,NaN,34.000000,NaN,NaN,NaN
50%,NaN,42.000000,NaN,NaN,NaN
75%,NaN,52.000000,NaN,NaN,NaN


## Construct final dataset

In [9]:
LABEL_COLS = ['sale_flg', 'sale_amount', 'contacts', 'profit', 'profitable']
CAT_FEATURES = ['gender', 'region', 'city', 'education']
# CAT_FEATURES = None

labels_all = funnel.set_index('client_id')[LABEL_COLS]
full_data = pd.concat([
    labels_all,
    balance_ft,
    client_ft
], axis=1)

## Split into train, val, test

In [10]:
train, val, test = train_val_test_split(full_data, [0.8, 0.1, 0.1])

train_pool = Pool(
    data = train.drop(LABEL_COLS, axis=1),
    label = train['profitable'],
    cat_features=CAT_FEATURES
)

val_pool = Pool(data = val.drop(LABEL_COLS, axis=1), cat_features=CAT_FEATURES)
test_pool = Pool(data = test.drop(LABEL_COLS, axis=1), cat_features=CAT_FEATURES)

print(val.profit.mean())
print(test.profit.mean())
print(train.profit.mean())

3043.2036976744193
2805.0767441860467
2501.6929265030817


## Construct model

In [11]:
model = CatBoostClassifier(
    iterations=1000,
    depth=10,
    loss_function='Logloss',
    custom_metric=['Accuracy'],
)

## Run training

In [12]:
model.fit(train_pool, verbose=100)

Learning rate set to 0.034712
0:	learn: 0.6247783	total: 109ms	remaining: 1m 48s
100:	learn: 0.2226168	total: 4.61s	remaining: 41s
200:	learn: 0.1981822	total: 8.75s	remaining: 34.8s
300:	learn: 0.1781326	total: 13.7s	remaining: 31.9s
400:	learn: 0.1564655	total: 19.3s	remaining: 28.8s
500:	learn: 0.1403387	total: 24.1s	remaining: 24s
600:	learn: 0.1271550	total: 29.7s	remaining: 19.7s
700:	learn: 0.1153494	total: 35.3s	remaining: 15.1s
800:	learn: 0.1046949	total: 40.9s	remaining: 10.2s
900:	learn: 0.0955748	total: 47.7s	remaining: 5.24s
999:	learn: 0.0876616	total: 53.7s	remaining: 0us


## Evaluate profitability of solution on val

In [13]:
pred_val = model.predict(val_pool)
bs_prof_val = val['profit'].mean()

model_prof_val = calculate_profit(val)[pred_val.astype(bool)].sum()
model_prof_val = model_prof_val / len(pred_val)

print(f'VAL | Baseline profit: {bs_prof_val:.2f}, model profit: {model_prof_val:.2f}')

VAL | Baseline profit: 3043.20, model profit: 3345.24


## Evaluate profitability of solution on test

In [14]:
pred_test = model.predict(test_pool)
bs_prof_test = test['profit'].mean()

model_prof_test = calculate_profit(test)[pred_test.astype(bool)].sum()
model_prof_test = model_prof_test / len(pred_test)

print(f'TEST | Baseline profit: {bs_prof_test:.2f}, model profit: {model_prof_test:.2f}')

TEST | Baseline profit: 2805.08, model profit: 3074.36


## Notes

1. Using just balance  or features, I get below baseline
2. Using balance + client features, I get 3074 on test

## Train model on FULL data and save it

In [ ]:
LABEL_COLS = ['sale_flg', 'sale_amount', 'contacts', 'profit', 'profitable']
CAT_FEATURES = ['gender', 'region', 'city', 'education']

labels_all = funnel.set_index('client_id')[LABEL_COLS]
full_data = pd.concat([
    labels_all,
    balance_ft,
    client_ft
], axis=1)

data_pool = Pool(
    data = full_data.drop(LABEL_COLS, axis=1),
    label = full_data['profitable'],
    cat_features=CAT_FEATURES
)

model.fit(data_pool, verbose=100)
model.save_model('models/tadej_model.cbm', 'cbm')

Learning rate set to 0.038183
0:	learn: 0.6212372	total: 51.3ms	remaining: 51.2s
100:	learn: 0.2266535	total: 6.5s	remaining: 57.9s
200:	learn: 0.2037682	total: 12.8s	remaining: 50.8s
300:	learn: 0.1828418	total: 18.6s	remaining: 43.3s
400:	learn: 0.1609730	total: 24.6s	remaining: 36.8s
500:	learn: 0.1458257	total: 31.1s	remaining: 31s
600:	learn: 0.1313589	total: 37.3s	remaining: 24.8s
700:	learn: 0.1190135	total: 43.4s	remaining: 18.5s
800:	learn: 0.1084753	total: 50.7s	remaining: 12.6s
900:	learn: 0.1001874	total: 57.1s	remaining: 6.27s


## Test load, data and produce predictions

In [ ]:
# Save data first, as it would be in the build step
LABEL_COLS = ['sale_flg', 'sale_amount', 'contacts', 'profit', 'profitable']
full_data.drop(LABEL_COLS, axis=1).to_pickle('final_version.pickle')

In [ ]:
model = CatBoostClassifier().load_model('models/tadej_model.cbm', 'cbm')
data = pd.read_pickle('final_version.pickle')
data_pool = Pool(data = data, cat_features=CAT_FEATURES)

target = model.predict(data_pool)
submission = pd.DataFrame({'client_id': data.index, 'target': target})
submission.to_csv('submission.csv', index=False)

In [ ]:
pd.read_csv('submission.csv')